In [7]:
from yelpapi import YelpAPI
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import time
import re

API_KEY = '3HutAvbl1TRuIXG0mjgNjTGPuLy_ZhoON7UEYzJqG6o2D6gPkIVZr1RT4x3TBpR3iXtJ9lBnis50H7MXRp0uSisapP-8Tup98qEipjCL6AjzLv2jHM6UhMXZN8IBYnYx'
yelp_api = YelpAPI(API_KEY)

In [8]:
# STEP 1: create list of neighborhoods for each midwest city
cities = ['Chicago, Illinois',
 'Columbus, Ohio', 
 'Indianapolis, Indiana',
 'Detroit, Michigan',
 'Milwaukee, Wisconsin',
 'Kansas City, Missouri',
 'St. Louis, Missouri',
 'Minneapolis, Minnesota',
 'Saint Paul, Minnesota']    # changed St. Paul to Saint Paul

# scraping wikipedia page
# https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests
page = ''
while page == '':
    try:
        page = requests.get('https://en.wikipedia.org/wiki/Lists_of_neighborhoods_by_city#United_States')
        break
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        continue
        
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
items = content.find_all('li')

In [9]:
# STEP 1.1: get directories (what comes after wikipedia base url) for all cities
# creating dict where keys are cities and values are directories
city_directories = {}

# iterate through list of cities on wikipedia page - if they match our cities, get its directory
for item in items:
    wiki_city_cleaned = item.text.strip('*').strip()
    if wiki_city_cleaned == 'Neighborhoods of Chicago':   # hard-coded
        city_directories[cities[0]] = item.a['href']
    for city in cities:
        if wiki_city_cleaned == city.split(',')[0] or wiki_city_cleaned == city: 
            city_directories[city] = item.a['href']

city_directories['Kansas City, Missouri'] = '/wiki/Neighborhoods_of_Kansas_City,_Missouri'   # hard-coded
city_directories

{'Chicago, Illinois': '/wiki/Neighborhoods_of_Chicago',
 'Columbus, Ohio': '/wiki/Neighborhoods_in_Columbus,_Ohio',
 'Detroit, Michigan': '/wiki/Neighborhoods_in_Detroit',
 'Indianapolis, Indiana': '/wiki/Neighborhoods_of_Indianapolis',
 'Milwaukee, Wisconsin': '/wiki/Neighborhoods_of_Milwaukee',
 'Minneapolis, Minnesota': '/wiki/Neighborhoods_of_Minneapolis',
 'Saint Paul, Minnesota': '/wiki/Neighborhoods_of_Saint_Paul',
 'St. Louis, Missouri': '/wiki/Neighborhoods_of_St._Louis',
 'Kansas City, Missouri': '/wiki/Neighborhoods_of_Kansas_City,_Missouri'}

In [10]:
# STEP 1.2: creating functions for scraping wikipedia page & cleaning
base_url = 'https://en.wikipedia.org'

# uses pandas read_html to scrape neighborhoods from table on each wiki page
def read_wiki(city):
    directory = city_directories.get(city)
    wiki = pd.read_html(base_url + directory)
    return wiki

# uses bs4 to scrape neighborhoods based on html tags
def soup(city):
    directory = city_directories.get(city)
    page = requests.get(base_url + directory)
    soup = BeautifulSoup(page.text, 'html.parser')
    content = soup.find('div', class_='mw-parser-output')
    return content

# cleans where neighborhoods have multiple names with '/'
# NOTE: still in progress

# def cleaned(lst):
#     new_lst = []
#     for neighborhood in lst:
#         if re.search(r"^[\w\s]+\s?[(\/\[]", neighborhood):
#             if "/" in neighborhood:
#                 towns = neighborhood.split("/")
#                 stripped_towns = [i.strip() for i in towns]
#                 new_lst.extend(stripped_towns)
#             elif "[" in neighborhood:
#                 matches = re.findall(r"^([\w\s]+)\s?[(\/\[]", neighborhood)
#                 stripped_matches = [i.strip() for i in matches]
#                 if matches:
#                     new_lst.extend(stripped_matches)
#             else:
#                 new_neighborhood = neighborhood.replace("(", "").replace(")", "").strip()
#                 new_lst.append(new_neighborhood)
#         else:
#             new_lst.append(neighborhood.strip())
#     return new_lst

def cleaned(lst):
    for neighborhood in lst:
        neighborhood = str(neighborhood)
        if '/' in neighborhood or ' / ' in neighborhood:
            lst.remove(neighborhood)
            aliases = neighborhood.split('/')
            for item in aliases:
                lst.append(item.strip())
        if '[' in neighborhood and neighborhood in lst:
            lst.remove(neighborhood)
            split = neighborhood.split('[')
            lst.append(split[0].strip())
    return lst

In [11]:
# STEP 1.3: get neighborhoods for chicago, indianapolis, detroit, st. louis
# creating dict where key is city and value is list of neighborhoods
neighborhoods = {}

# scraping chicago neighborhoods
wiki_chicago = read_wiki('Chicago, Illinois')
chicago_neighborhoods = wiki_chicago[0]
chicago_neighborhoods = list(chicago_neighborhoods['Neighborhood'])
neighborhoods['Chicago, Illinois'] = cleaned(chicago_neighborhoods)

# scraping indianapolis neighborhoods
wiki_ind = read_wiki('Indianapolis, Indiana')
ind_neighborhoods = wiki_ind[2]
indianapolis_neighborhoods = list(ind_neighborhoods[ind_neighborhoods.columns[0]])
neighborhoods['Indianapolis, Indiana'] = cleaned(indianapolis_neighborhoods)

# scraping st. louis neighborhoods
wiki_stl = read_wiki('St. Louis, Missouri')
stl_neighborhoods = wiki_stl[1]
stl_neighborhoods = list(stl_neighborhoods['Neighborhood'])
neighborhoods['St. Louis, Missouri'] = cleaned(stl_neighborhoods)

# scraping detroit neighborhoods
wiki_detroit =  read_wiki('Detroit, Michigan')
detroit_neighborhoods = []
for page in wiki_detroit:
    if 'Name' in page.columns:
        detroit_neighborhoods.extend(list(page['Name']))
        
# cleaning detroit neighborhoods
detroit_neighborhoods = cleaned(detroit_neighborhoods)
for n in detroit_neighborhoods:
    if '/' in n:
        detroit_neighborhoods.remove(n)
        names = n.split('/')
        for name in names:
            detroit_neighborhoods.append(name)
neighborhoods['Detroit, Michigan'] = detroit_neighborhoods

In [12]:
# STEP 1.4: get neighborhoods for columbus, milwaukee, kansas city, minneapolis, saint paul 
# NOTE: need fix repetition of code later for efficiency

# scraping columbus neighborhoods
columbus_directory = city_directories.get('Columbus, Ohio')
page = requests.get(base_url + columbus_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
columbus_neighborhoods = [item.text.replace('[edit]', '') for item in content.find_all(['h2', 'h3', 'h4'])][2:-3]
neighborhoods['Columbus, Ohio'] = cleaned(columbus_neighborhoods)

# scraping milwaukee neighborhoods
mlwk_directory = city_directories.get('Milwaukee, Wisconsin')
page = requests.get(base_url + mlwk_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
mlwk_neighborhoods = [item.text.replace('[edit]', '').strip('"') for item in content.find_all('h3')]
neighborhoods['Milwaukee, Wisconsin'] = cleaned(mlwk_neighborhoods)

# scraping kansas neighborhoods
kansas_directory = city_directories.get('Kansas City, Missouri')
page = requests.get(base_url + kansas_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
li_list = [item.text for item in content.find_all('li')]
kansas_neighborhoods = li_list[li_list.index('CBD-Downtown'):li_list.index('Wornall Homestead') + 1]
neighborhoods['Kansas City, Missouri'] = cleaned(kansas_neighborhoods)

# scraping minneapolis neightborhoods
minneapolis_directory = city_directories.get('Minneapolis, Minnesota')
page = requests.get(base_url + minneapolis_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
h3 = [item.text.replace('[edit]', '').strip('"') for item in content.find_all(['h3'])]
li = [item.text for item in content.find_all(['li']) if item.text[0].isalpha()]
li = li[0:li.index('University') + 1]
b = [item.text for item in content.find_all('b') if item.text[0].isalpha() and item.text not in li]
minneapolis_neighborhoods = h3 + li + b
neighborhoods['Minneapolis, Minnesota'] = cleaned(minneapolis_neighborhoods)

# scraping saint paul neighborhoods
stpaul_directory = city_directories.get('Saint Paul, Minnesota')
page = requests.get(base_url + stpaul_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
h3_list = [item.text.strip().split('- ')[1].replace('[edit]', '') for item in content.find_all('h3')]
stpaul_neighborhoods = []
for h3 in h3_list:
    names = h3.split(',')
    for name in names:
        stpaul_neighborhoods.append(name.strip())
neighborhoods['Saint Paul, Minnesota'] = cleaned(stpaul_neighborhoods)

In [13]:
# step 2: use yelp API to get restaurant data for each neighborhood and write results into csv file
# starting with detroit

# restaurant_ids = []

with open('detroit_restaurants_test.csv', "w", newline="") as fileout:
    header = fileout.write("id,name,review_count,rating,latitude,longitude,price,zip_code,neighborhood\n")
    writer = csv.writer(fileout)
    for n in neighborhoods['Detroit, Michigan']:
        offset = 0
        try:
            while offset <= 500:
                result = yelp_api.search_query(categories='restaurants', location=n + ', Detroit', limit=50, offset=offset)['businesses']
                for business in result:
                    id = business['id']
                    name = business['name']
                    review_count = business['review_count']
                    rating = business['rating']
                    latitude = business['coordinates']['latitude']
                    longitude = business['coordinates']['longitude']
                    zip_code = business['location']['zip_code']
                    try:
                        price = business['price']
                    except KeyError:
                        price = 'NA'
                    row = id,name,review_count,rating,latitude,longitude,price,zip_code,n
                    writer.writerow(row)
                    print(name + ' (' + n + ')')
                offset += 50
                    
#                         print('Writing for ' + n + ' - ' + str(offset))
#                     if id not in restaurant_ids:
#                         row = id,name,review_count,rating,latitude,longitude,price,zip_code,n
#                         writer.writerow(row)
#                         offset += 1
#                         print('Writing for ' + n + ' - ' + str(offset))
#                         restaurant_ids.append(id)
#                     else:
#                         print('Skipping duplicates...')
#                         continue
        except YelpAPI.YelpAPIError as e:
            print('Error')
            continue
    print('Done')

Jacoby's German Biergarten (Bricktown Historic District)
Sweetwater Tavern (Bricktown Historic District)
Niki's Pizza (Bricktown Historic District)
Detroiter Bar (Bricktown Historic District)
Floods Bar & Grille (Bricktown Historic District)
The Elephant Room (Bricktown Historic District)
Pegasus Taverna - Greektown (Bricktown Historic District)
Red Smoke Barbecue (Bricktown Historic District)
Loco's Tex-Mex Grille (Bricktown Historic District)
The Clique Restaurant (Bricktown Historic District)
Grand Trunk Pub (Bricktown Historic District)
Basement Burger Bar (Bricktown Historic District)
Central Kitchen + Bar (Bricktown Historic District)
Nathan's Deli (Bricktown Historic District)
Prism (Bricktown Historic District)
Firebird Tavern (Bricktown Historic District)
Townhouse (Bricktown Historic District)
Wahlburgers (Bricktown Historic District)
Besa (Bricktown Historic District)
Golden Fleece Restaurant (Bricktown Historic District)
Fishbone's (Bricktown Historic District)
Highlands (B

Dime Store (Campus Martius Park)
Wright & Company (Campus Martius Park)
Lafayette Coney Island (Campus Martius Park)
The Hudson Cafe (Campus Martius Park)
Standby (Campus Martius Park)
Green Dot Stables (Campus Martius Park)
Supino Pizzeria (Campus Martius Park)
Mudgie's Deli & Wine Shop (Campus Martius Park)
Selden Standard (Campus Martius Park)
Vicente's Cuban Cuisine (Campus Martius Park)
Parc (Campus Martius Park)
Roasting Plant Detroit (Campus Martius Park)
Grey Ghost Detroit (Campus Martius Park)
SavannahBlue (Campus Martius Park)
Batch Brewing Company (Campus Martius Park)
Andiamo (Campus Martius Park)
The Apparatus Room (Campus Martius Park)
Ottava Via (Campus Martius Park)
Maru Sushi & Grill Detroit (Campus Martius Park)
Vertical Detroit (Campus Martius Park)
Joe Muer Seafood (Campus Martius Park)
Cornerstone Barrel House (Campus Martius Park)
ima (Campus Martius Park)
Bakersfield (Campus Martius Park)
Slows Bar BQ (Campus Martius Park)
Cannelle Matt Knio (Campus Martius Park)

Monroe Market (Capitol Park Historic District)
Jimmy John's (Capitol Park Historic District)
Detroit Sports Bar and Grille (Capitol Park Historic District)
Panera Bread (Capitol Park Historic District)
Crazy Gringo Express (Capitol Park Historic District)
UWM District Market (Capitol Park Historic District)
Town Pump Tavern (Capitol Park Historic District)
Forty-Two Degrees North (Capitol Park Historic District)
Aratham (Capitol Park Historic District)
Burger King (Capitol Park Historic District)
Downtown Loius (Capitol Park Historic District)
Tim Horton's (Capitol Park Historic District)
Three Legged Goat (Capitol Park Historic District)
Coney Town (Capitol Park Historic District)
313 Burger Bar (Capitol Park Historic District)
Dagwood's Deli & Diner (Capitol Park Historic District)
Heavenly Chicken and Waffles (Capitol Park Historic District)
Pig's Tail (Capitol Park Historic District)
Subway Restaurants (Capitol Park Historic District)
The XO Room (Capitol Park Historic District)
Ri

Monroe Market (Detroit Financial District)
Jimmy John's (Detroit Financial District)
Detroit Sports Bar and Grille (Detroit Financial District)
Panera Bread (Detroit Financial District)
Crazy Gringo Express (Detroit Financial District)
UWM District Market (Detroit Financial District)
Town Pump Tavern (Detroit Financial District)
Forty-Two Degrees North (Detroit Financial District)
Aratham (Detroit Financial District)
Burger King (Detroit Financial District)
Downtown Loius (Detroit Financial District)
Tim Horton's (Detroit Financial District)
Three Legged Goat (Detroit Financial District)
Coney Town (Detroit Financial District)
313 Burger Bar (Detroit Financial District)
Dagwood's Deli & Diner (Detroit Financial District)
Heavenly Chicken and Waffles (Detroit Financial District)
Pig's Tail (Detroit Financial District)
Subway Restaurants (Detroit Financial District)
The XO Room (Detroit Financial District)
Rice Bowl Express (Detroit Financial District)
Qzine Cafe (Detroit Financial Distr

Jimmy John's (Greektown)
Panera Bread (Greektown)
Subway (Greektown)
M&M Food Corner (Greektown)
UWM District Market (Greektown)
Terrace Cafe (Greektown)
Heavenly Chicken and Waffles (Greektown)
Detroit Sports Bar and Grille (Greektown)
Domino's Pizza (Greektown)
Burger King (Greektown)
Town Pump Tavern (Greektown)
Papa's Pizza & BBQ (Greektown)
Crazy Gringo Express (Greektown)
Stack'd (Greektown)
Coney Town (Greektown)
Tim Horton's (Greektown)
Mo Better Blues (Greektown)
Rice Bowl Express (Greektown)
Gyro Land (Greektown)
Downtown Loius (Greektown)
Aratham (Greektown)
Diamond Dogs (Greektown)
Dagwood's Deli & Diner (Greektown)
Domino's Pizza (Greektown)
Subway Restaurants (Greektown)
The XO Room (Greektown)
Qzine Cafe (Greektown)
Subway (Greektown)
River Cafe (Greektown)
Motor City Pantry (Greektown)
Zef's Coney Island (Greektown)
The Boulevard Room (Greektown)
That Sandwich Place (Greektown)
Hungry Howie's Pizza (Greektown)
Bohemia (Greektown)
Olin (Greektown)
Tropical Smoothie Cafe 

...

Papa's Pizza & BBQ (North End)
Cakes and Shakes (North End)
Fresh Fish House (North End)
Legend's Coney Island West (North End)
Jet's Pizza (North End)
Beyond Juicery + Eatery (North End)
Sun Sun Restaurant (North End)
Lila's Diner (North End)
Byblos Cafe & Grill II (North End)
Delite Cafe Deli Detroit (North End)
SuperCrisp (North End)
Pink Flamingo To Go (North End)
Heavenly Chicken and Waffles (North End)
Coney Man (North End)
Barcade (North End)
Ham Palace (North End)
Papa Joe's Market (North End)
BIGGBY COFFEE (North End)
Asian Corned Beef (North End)
Boostan Cafe Detroit (North End)
Simply Spanish (North End)
Street Beet (North End)
Checkers (North End)
Jolly Old Timers (North End)
US Fish and Chicken (North End)
Small World Kitchen (North End)
Burger King (North End)
Sisters on a Roll Mobile Cafe (North End)
Subway Restaurants (North End)
Wing Fellas (North End)
Value Pizza (North End)
Jimmy John's (North End)
Ray’s Coney Island & Pizza (North End)
Joe Ann's Bar B-Q (North End)


Al Haramain International Foods (Osborn)
Prince Pizza (Osborn)
Lape Soulfood (Osborn)
Stosh's Pizza (Osborn)
Peking Chinese Restaurant (Osborn)
House of FusYin (Osborn)
Seng Thai (Osborn)
Shredderz Food Truck (Osborn)
The Turkey Grill (Osborn)
Classic Coney Grill (Osborn)
Rabauts Cafe (Osborn)
Nonos Pizza (Osborn)
Cattleman’s Meat (Osborn)
Antonio's in the Park (Osborn)
Ivy Kitchen + Cocktails (Osborn)
Kwong Tung Inn (Osborn)
Legend's Coney Express (Osborn)
Royal Barbecue Pizza (Osborn)
5 Star Restaurant (Osborn)
Pepperoni Grille (Osborn)
Bellacino's Pizza & Grinders (Osborn)
Blue Finn Bar & Grill (Osborn)
Te Roma (Osborn)
Village Grille (Osborn)
Jersey Mike's Subs (Osborn)
Shawarma Express Eastpointe (Osborn)
Five Guys (Osborn)
Cozy Cabin (Osborn)
Lou's Coney Time (Osborn)
Beyond Juicery + Eatery (Osborn)
The Big Salad (Osborn)
Kim Toa Restaurant (Osborn)
Milestone Grill (Osborn)
Mayflower Chinese Gourmet Restaurant (Osborn)
Bonoful Sweets & Cafe (Osborn)
Del Taco (Osborn)
Sabby's Bar

Terrace Cafe (Lafayette Park)
Burger King (Lafayette Park)
McDonald's (Lafayette Park)
313 Burger Bar (Lafayette Park)
Planted Detroit (Lafayette Park)
Happy's Pizza (Lafayette Park)
Pig's Tail (Lafayette Park)
Three Legged Goat (Lafayette Park)
Domino's Pizza (Lafayette Park)
Coney Town (Lafayette Park)
Crazy Gringo Express (Lafayette Park)
Geisha Girls (Lafayette Park)
Little Caesars Pizza (Lafayette Park)
Rice Bowl Express (Lafayette Park)
Gyro Land (Lafayette Park)
Kabob N Grill (Lafayette Park)
Downtown Loius (Lafayette Park)
Mo Better Blues (Lafayette Park)
Aratham (Lafayette Park)
Burger King (Lafayette Park)
Tim Horton's (Lafayette Park)
Hungry Howie's Pizza (Lafayette Park)
Church's Chicken (Lafayette Park)
Tacos Te Amo (Lafayette Park)
Gratiot Grill (Lafayette Park)
River Cafe (Lafayette Park)
Motor City Pantry (Lafayette Park)
Qzine Cafe (Lafayette Park)
Stack'd (Lafayette Park)
Zef's Coney Island (Lafayette Park)
Domino's Pizza (Lafayette Park)
Subway Restaurants (Lafayette

Dime Store (NoHam)
Green Dot Stables (NoHam)
Selden Standard (NoHam)
Chartreuse (NoHam)
Slows Bar BQ (NoHam)
Supino Pizzeria (NoHam)
Wright & Company (NoHam)
Mudgie's Deli & Wine Shop (NoHam)
Lafayette Coney Island (NoHam)
Grey Ghost Detroit (NoHam)
Takoi (NoHam)
Taqueria El Rey (NoHam)
The Hudson Cafe (NoHam)
ima (NoHam)
Standby (NoHam)
Batch Brewing Company (NoHam)
Ottava Via (NoHam)
Royale with Cheese (NoHam)
Vicente's Cuban Cuisine (NoHam)
Pie Sci (NoHam)
Bakersfield (NoHam)
Motor City Brewing Works-Canfield (NoHam)
Taqueria Mi Pueblo (NoHam)
Johnny Noodle King (NoHam)
Buddy's Pizza (NoHam)
Avalon International Breads (NoHam)
SavannahBlue (NoHam)
Mercury Burger Bar (NoHam)
Honey Bee La Colmena (NoHam)
Andiamo (NoHam)
Chili Mustard Onions (NoHam)
Giovanni's Ristorante (NoHam)
Bobcat Bonnie's (NoHam)
SheWolf Pastificio & Bar (NoHam)
Parc (NoHam)
Joe Muer Seafood (NoHam)
The Apparatus Room (NoHam)
Roasting Plant Detroit (NoHam)
Pupusería y Restaurante Salvadoreño (NoHam)
Urban Ramen (

The Cook's Shop (NoHam)
Hockeytown Cafe (NoHam)
Voyager (NoHam)
The Turkey Grill (NoHam)
Loco's Tex-Mex Grille (NoHam)
La Terraza (NoHam)
Kidcadia Play Café (NoHam)
Los Corrales Restaurant (NoHam)
Chicago's Pizza (NoHam)
Detroit One Coney Island (NoHam)
Bangkok Cafe (NoHam)
Culantro (NoHam)
Bobcat Bonnie's Ferndale (NoHam)
Steve's Soul Food (NoHam)
The Lobster Food Truck (NoHam)
Starter's Bar and Grill (NoHam)
Mi Pueblo Express (NoHam)
Chick-fil-A (NoHam)
Dos Locos Tacos (NoHam)
The Drunken Rooster (NoHam)
James Oliver Coffee (NoHam)
The Carvery (NoHam)
Connie & Barbara's Soul Food Restaurant (NoHam)
Bishr Poultry & Food Center (NoHam)
Now Cafe (NoHam)
Elwood Bar & Grill (NoHam)
Athens Souvlaki (NoHam)
Detroit Fleat (NoHam)
Aladdin Sweets & Cafe (NoHam)
Wigley's Meats & Produce (NoHam)
Mocad Café (NoHam)
Urban Soul (NoHam)
The Taco Stand Taqueria (NoHam)
Dynasty Chinese Food (NoHam)
COOP Carribean Fusion (NoHam)
Halal Desi Pizza & Gyro's (NoHam)
Howe's Bayou (NoHam)
Ukrainian Restauran